# 概括
* 只交易一次 -- 不限交易次数 -- 只交易两次 -- 加入‘冷冻期’和‘手续费’
* 解题关键：选择‘状态’
* 状态机
* 每天都有三种‘选择’：买入·卖出·无操作   
    条件：sell必须在buy之后，buy必须在sell之后，rest的状态有两种：一种是buy之后的持股，另一种是sell之后的观望
* 这个问题的状态有三个：   
    1. 天数
    2. 允许交易的最大次数
    3. 当前持有状态，0表示观望（不持股），1表示持股

In [ ]:
n是天数，K是最多交易次数，0和1表示不持股状态和持股状态
此问题共有n * K * 2种状态
dp[i][k][0 or 1]表示当前状态下的利润

0 <= i <= n-1, 1 <= k <= K
dp[i][k][0 or 1]
比如：dp[3][2][1] 的含义就是：今天是第三天，我现在手上持有着股票，至今最多进行 2 次交易。值表示利润？
再比如 dp[2][3][0] 的含义：今天是第二天，我现在手上没有持有股票，至今最多进行 3 次交易

for i in range(n-1):
    for k in range(1, K + 1):
        for s in [0, 1]:
            dp[i][k][s] = max(buy, sell, rest)
            
最终答案是： dp[n - 1][K][0]，即最后一天，最多允许 K 次交易，最多获得多少利润
# 为什么不是 dp[n - 1][K][1]？
# 因为 [1] 代表手上还持有股票，[0] 表示手上的股票已经卖出去了，很显然后者得到的利润一定大于前者。

# 状态转移方程式
dp[i][k][0] = max(选择rest， 选择sell)
            = max(dp[i-1][k][0], dp[i-1][k][1] + price[i])
            # 今天没有股，要么就是昨天就没有股，要么就是今天sell了，利润里面增加price[i]
    
dp[i][k][1] = max(选择rest， 选择buy)
            = max(dp[i-1][k][1], dp[i-1][k-1][0] - price[i])
            # 今天有股，要么就是昨天就有股，要么就是今天buy了，利润里面减少price[i]
        
# base case
dp[-1][k][0] = 0
    # i = -1，意味着还没有开始，利润是0
dp[-1][k][1] = -infinity
    # 还没有开始的时候，是不可能持有股票，用负无穷大表示不可能 不好理解
dp[i][0][0] = 0
    # k = 0, 从未交易过
dp[i][0][1] = -infinity
    # 不允许交易的情况下，是不可能持有股票，用负无穷大表示不可能

# 题目
## 只交易一次
* 121


In [ ]:
K = 1, 不会发生改变，所以可以简化成dp[i][0 or 1]

# 状态转移方程式：
dp[i][0] = max(选择rest， 选择sell)
         = max(dp[i-1][0], dp[i-1][1] + price[i])
dp[i][1] = max(选择rest， 选择buy)
         = max(dp[i-1][1], dp[i-1][0] - price[i]) # 错误！！！why??? 
                                                  # 因为这样写表示i-1天的时候，没有持股，拥有dp[i-1][0]元的利润，再买入新股时，-price[i]; 可是只有一次买卖，i-1天的时候，没有持股也就表示没有过交易，现在的利润为0
         = max(dp[i-1][1], - price[i]) # 因为只有一次买卖，买进就表示-price
# base：
dp[0][0] = 0
    # 第0天，还没有持股
dp[0][1] = -infinity
    # 用无穷大表示不可能存在

In [ ]:
class Stock:
    def maxProfit(self, prices):
        if prices == []:
            return 0
        
        n = len(prices)
        dp = [[float('-inf')] * 2 for _ in range(n + 1)]
        dp[0][0] = 0

        for i in range(1, n+1):
            dp[i][0] = max(dp[i-1][0], dp[i-1][1] + prices[i-1])  # prices[i-1]易错
            dp[i][1] = max(dp[i-1][1], - prices[i-1])  # prices[i-1]易错
        return dp[n][0]

'''
状态转移方程，新状态只和相邻的一个状态有关，其实不用整个 dp 数组，
只需要一个变量储存相邻的那个状态就足够了，这样可以把空间复杂度降到 O(1)
'''

class Stock:
    def maxProfit(self, prices):
        if prices == []:
            return 0
        
        n = len(prices)
        dp_0 = 0
        dp_1 = float('-inf')
        for i in range(n):
            dp_0 = max(dp_0, dp_1 + prices[i])
            dp_1 = max(dp_1, -prices[i])
        return dp_0

## 交易次数不限制
* 122


In [ ]:
k是任意次数，所以不需要记录k这个状态
# 状态转移方程式
dp[i][0] = max(选继续观望， 选卖掉股票)
         = max(dp[i-1][0], dp[i-1][1] + price(i))

dp[i][1] = max(选继续持股， 选买入股票)
         = max(dp[i-1][1], dp[i-1][0] - price[i])  # 121题与122题的区别

# base
dp[0][0] = 0
    # 第0天，还没有持股
dp[0][1] = -infinity
    # 用无穷大表示不可能存在

In [ ]:
class Stock:
    def maxProfit(self, prices):
        if prices == []:
            return 0
        
        n = len(prices)
        dp_0 = 0
        dp_1 = float('-inf')

        for i in range(n):
            dp_0 = max(dp_0, dp_1 + prices[i])
            dp_1 = max(dp_1, dp_0 - prices[i])
        return dp_0

## 交易次数被规定
* 123
* 交易2次

In [ ]:

k == 0
没有任何交易，都是0
k == 1,2
持股或观望

# 状态转移方程式
dp[i][k][0] = max(dp[i-1][k][0], dp[i-1][k][1] + prices[i])
dp[i][k][1] = max(dp[i-1][k][1], dp[i-1][k-1][0] - prices[i])

# base
dp[-1][k][0] = 0
dp[-1][k][1] = -infinity
dp[i][0][0] = 0
dp[i][0][1] = -infinity


In [ ]:
class Stock:
    def maxProfit(self, prices):
        if prices == []:
            return 0

        n = len(prices)
        max_k = 2
        dp = [[[float('-inf')] * 2 for _ in range(max_k + 1)] for _ in range(n+1)]

        for i in range(0, n + 1):
            for k in range(0, max_k + 1):
                if i == 0:
                    dp[0][k][0] = 0
                elif k == 0:
                    dp[i][0][0] = 0
                else:
                    dp[i][k][0] = max(dp[i-1][k][0], dp[i-1][k][1] + prices[i-1])
                    dp[i][k][1] = max(dp[i-1][k][1], dp[i-1][k-1][0] - prices[i-1])
        return dp[n][max_k][0]

In [ ]:
# 方法2
# 这里的k是2
# 可以把k的范围直接列出来

# 状态转移方程式
dp[i][2][0] = max(dp[i-1][2][0], dp[i-1][2][1] + prices[i])
dp[i][2][1] = max(dp[i-1][2][1], dp[i-1][1][0] - prices[i])
dp[i][1][0] = max(dp[i-1][1][0], dp[i-1][1][1] + prices[i])
dp[i][1][1] = max(dp[i-1][1][1], -prices[i])

# base
dp[-1][k][0] = 0
dp[-1][k][1] = -infinity
dp[i][0][0] = 0
dp[i][0][1] = -infinity

In [ ]:
class Stock:
    def maxProfit(self, prices):
        if prices == []:
            return 0
            
        n = len(prices)
        dp_i10 = 0
        dp_i20 = 0
        dp_i11 = float('-inf')
        dp_i21 = float('-inf')

        for i in prices:
            dp_i20 = max(dp_i20, dp_i21 + 1)
            dp_i21 = max(dp_i21, dp_i10 - 1)
            dp_i10 = max(dp_i10, dp_i11 + 1)
            dp_i11 = max(dp_i11, -1)

        return dp_i20

## 不限交易次数，有一天的冻结
* After you sell your stock, you cannot buy stock on next day. (ie, cooldown 1 day)
* 309

In [ ]:
k = 任意
After you sell your stock, you cannot buy stock on next day. (ie, cooldown 1 day)

# 状态转移方程
dp[i][0] = max(继续观望，  卖掉)
         = max(dp[i-1][0], dp[i-1][1] + prices[i])
dp[i][1] = max(继续持股，  买入)
         = max(dp[i-1][1], dp[i-2][0] - prices[i])

# base
dp[0][0] = 0
dp[0][1] = -inf
dp[1][1] = - prices[1]

In [ ]:
class Solution:
    def maxProfit(self, prices: List[int]) -> int:
        if prices == []:
            return 0
        
        n = len(prices)
        dp = [[-float('inf')] * 2 for _ in range(n+1)]
        
        dp[0][0] = 0
        
        
        for i in range(1, n + 1):
            if i == 1:
                dp[i][0] = max(dp[i-1][0], dp[i-1][1] + prices[i-1])
                dp[i][1] = max(dp[i-1][1], - prices[i-1])
            else:
                dp[i][0] = max(dp[i-1][0], dp[i-1][1] + prices[i-1])
                dp[i][1] = max(dp[i-1][1], dp[i-2][0] - prices[i-1])
        return dp[n][0]
        

## 不限次数，有手续费
* 714

In [ ]:
k 任意
卖掉会有fee

# 状态转移方程
dp[i][0] = max(继续观望，  卖掉)
         = max(dp[i-1][0], dp[i-1][1] + prices[i] - fee)
dp[i][1] = max(继续持股，  买入)
         = max(dp[i-1][1], dp[i-1][0] - prices[i])
         
# base
dp[0][0] = 0
dp[0][1] = - prices[0]

In [ ]:
class Solution:
    def maxProfit(self, prices: List[int], fee: int) -> int:
        if prices == None:
            return 0
        
        dp_i_0 = 0
        dp_i_1 = - prices[0]
        
        for i in range(len(prices)):
            dp_i_0 = max(dp_i_0, dp_i_1 + prices[i] - fee)
            dp_i_1 = max(dp_i_1, dp_i_0 - prices[i]) 
        return dp_i_0

## 指定K
* 188题
* 一次交易由买入和卖出构成，至少需要两天。所以说有效的限制 k 应该不超过 n/2，如果超过，就没有约束作用了，相当于 k = +infinity。这种情况是之前解决过的
* 122题+123题

# Python

In [1]:
[0] * 2

[0, 0]

In [2]:
b = [[0] for _ in range(2)]
b

[[0], [0]]

In [5]:
c = [[0] * 2 for _ in range(2)]
c

[[0, 0], [0, 0]]

In [8]:
d = [[[0] * 2 for _ in range(2)] for _ in range(3)]
d

[[[0, 0], [0, 0]], [[0, 0], [0, 0]], [[0, 0], [0, 0]]]

In [ ]:
dp = [[[-float('inf'), -float('inf')] for _ in range(k + 1)] for _ in range(n+1)]  # 易写错
